In [25]:
library(COMPoissonReg)
library(boot)
library(CompGLM)

In [2]:
data.raw = read.table("crabs.dat.txt", header = TRUE)
data.raw = data.raw[, 2:7]
data.raw$color = factor(data.raw$color)
data.raw$y = factor(data.raw$y)

In [52]:
cmp.color = glm.cmp(spine ~ color, data = data.raw)

In [53]:
print(cmp.color)

Fit for CMP coefficients
              Estimate     SE z.value   p.value
X:(Intercept)   2.6204 0.5778  4.5349 5.762e-06
X:color2        2.2335 0.5444  4.1031 4.076e-05
X:color3        2.7977 0.5856  4.7775 1.775e-06
X:color4        2.9293 0.6185  4.7359 2.181e-06
S:(Intercept)   1.5442 0.1095 14.0988 3.865e-45
--
Transformed intercept-only parameters
   Estimate     SE
nu   4.6843 0.5131
--
Chi-squared test for equidispersion
X^2 = 116.6616, df = 1, p-value = 3.4048e-27
--
Elapsed Sec: 2.67   Sample size: 173
LogLik: -202.0806   AIC: 414.1612   BIC: 429.9277   


In [41]:
data.raw[, 'color'] = as.factor(data.raw[, 'color'])
cmp.color = glm.comp(spine ~ color, data = data.raw[, c('spine', 'color')])
df = data.frame(color = data.raw[, c('color')])
# df[, 1] = as.factor(df[, 1])
pred = (predict(cmp.color, newdata = df))


comp_mean = function(lambda, nu){
    Z = 0
    for (i in c(1:100)){
        Z = Z + (lambda^i)/(factorial(i)^nu)
    }
    average = 0
    for (j in c(1:100)){
        average = average + (j*(lambda^j))/(factorial(j)^nu)
    }
    return (average/Z)
}

comp_mean(pred[1,1], pred[1,2])


lambda 
2.412315

[1] 3


In [57]:
data.color = data.raw[, c('spine', 'color')]
folds <- cut(seq(1,nrow(data.color)),breaks=10,labels=FALSE)
error = 0
#Perform 10 fold cross validation
print(1)
for(i in 1:10){
    err = 0
    #Segement your data by fold using the which() function 
    testIndexes <- which(folds==i,arr.ind=TRUE)
    testData <- data.color[testIndexes, ]
    trainData <- data.color[-testIndexes, ]

    model = glm.comp(spine ~ color, data = trainData[, c('spine', 'color')])
    print(1)
    pred = predict(model, newdata = testData)
    
    
    for (i in c(1:nrow(pred))){
        err = err + (testData[i, 6] - as.numeric(comp_mean(pred[i,1], pred[i,2])))^2
    }
    
    error = error + err/nrow(pred)
}

error/10

[1] 1
[1] 1


Warning message in xLam %*% beta + lamOffset:
"longer object length is not a multiple of shorter object length"

ERROR: Error in predict.Comp(model, newdata = testData): dims [product 18] do not match the length of object [155]


In [58]:
testData

spine,color
3,2
3,3
1,1
3,3
3,3
3,2
1,1
2,3
1,2
3,3
